In [1]:
# 1) import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# 2) Create a path to chrome driver: Make sure to change the path!
executable_path = {'executable_path': 'chromedriver'}
# 3) load the path into browser 4) initialize the browser
browser = Browser('chrome', **executable_path, headless=True)


In [3]:
###############################################################
# EXTRACTION WITH TRANSFORMATION:
###############################################################

#FRYS:
###############################################################
# 5) initialize the page parameters and the first site visit:
###############################################################
next_page = "https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage=0&start=0&rows=20"
page_remaining = True
page_no = 0
increment = 20
start_no = 0
# 6) make a variable that holds the url we want to scrape data from
laptops = {}

frys_laptops = []

# 7) using the browser function, visit the webpage
browser.visit(next_page)
# 8) stop the scrape process for 1.25 seconds using the time function. This allows the webpage to load in full
time.sleep(1.25)

# 9) create the html variable that allows the browser to visit the HTML elements on the webpage
html = browser.html
# 10) initialize the BeautifulSoup module by telling it to parse html from the HTML elements on the webpage.
soup = bs(html, 'html.parser')


# 11) loop through until no products are found:
###############################################################
while page_remaining == True:
    
    # 12) Data retrieval logic: 
    ###############################################################
    try:
        mothers = soup.find_all('div', class_='togrid')
        for mother in mothers:
            title = mother.find('p', class_='productDescp').get_text().strip()
            
            raw_price = mother.find('label', class_='red_txt').get_text().strip()
            text_price = raw_price.replace("$", "")
            nocommas = text_price.replace(",", "")
            price = float(nocommas)
            
            partial_link = mother.find('p', class_='productDescp').find('a')['href'].strip()
            base_url = "https://www.frys.com/"
            link = base_url + partial_link
            
            results = mother.find_all('div', class_='prodModel')
            for result in results:
                new_list = result.find_all('p', class_='mar-btm')
                for p in new_list:
                    dataitem = p.text.replace(" ", "")
                    di = dataitem.replace('\n', "")
                    dataitem_split = di.split(":")
                    if dataitem_split[0] == "Brand":
                        brand = dataitem_split[1]
                    if dataitem_split[0] == "UPC":
                        upc = dataitem_split[1]
                    if dataitem_split[0] == "Model":
                        model = dataitem_split[1]
                        
            # 13) create a new dictionary that will hold the data that we scrape
            dic = {
                'title': title,
                'price': price,
                'link': link,
                'upc': upc,
                'model': model,
                'brand': brand
            }
            print(dic)
            frys_laptops.append(dic)
            
    except AttributeError:
        # errors out when there are 2 prices in red (price with rebate)
        print("error found. skipping...")
    
    
    # 14) then process the Next Page logic:
    ###############################################################
    page_no += 1
    start_no = start_no + increment
    next_page = f'https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage={page_no}&start={start_no}&rows=20'

    browser.visit(next_page)
    time.sleep(1.25)

    html = browser.html
    soup = bs(html, 'html.parser') 
    
    next_page_avail = soup.find_all('p')
    for p in next_page_avail:
        message = p.text
        if message == " No products were found that matched your search":
            page_remaining = False
        


{'title': 'Dell E6230 12.5" Refurbished Laptop With Intel Ci5-3320m Processor, 4GB Memory, 250GB Hard Drive, Windows 10', 'price': 399.0, 'link': 'https://www.frys.com/product/9907916;jsessionid=0CC07A35E80033C587B7DB861471027F.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897763632', 'model': 'E6230i5-3320m', 'brand': 'Dell'}
{'title': 'Lenovo Refurbished Thinkpad Yoga 11E 11.6" Touchscreen 2-in-1 Laptop With Intel Celeron N2930 Processor, 4GB memory, 320GB HD and Windows10 Home', 'price': 349.0, 'link': 'https://www.frys.com/product/9686093;jsessionid=0CC07A35E80033C587B7DB861471027F.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897736896', 'model': 'ThinkpadYoga11E', 'brand': 'Lenovo'}
{'title': 'HP PB 6470B 14" Refurbished Laptop With Intel Ci5-3210M Processor, 8GB Memory, 500GB Hard Drive, DVD-ROM, Windows 10', 'price': 399.0, 'link': 'https://www.frys.com/product/9800005;jsessionid=0CC07A35E80033C587B7DB861471027F.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897751172', 'mode

{'title': 'Acer A315-21-90LC 15.6" Laptop With AMD A9-9420e Processor, 12GB Memory, 1TB Hard Drive, Windows 10', 'price': 479.0, 'link': 'https://www.frys.com/product/9871315?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '193199075475', 'model': 'A315-90LCA9-9420e', 'brand': 'Acer'}


ValueError: could not convert string to float: '1,999.00'

In [ ]:
# BESTBUY:
###############################################################
next_url = "https://www.bestbuy.com/site/windows/windows-laptops/pcmcat748300547900.c?id=pcmcat748300547900"

page_remaining = True
page_no = 1

laptops = {}
bestbuy_laptops = []

browser.visit(next_url)
time.sleep(1.5)

html = browser.html
soup = bs(html, 'html.parser')


while page_remaining == True:
    try:
        mothers = soup.find_all('div', class_='right-column')
        for mother in mothers:
            title = mother.find('div', class_='sku-title').get_text().strip()
            
            old_price = mother.find('div', class_='priceView-hero-price priceView-customer-price').find('span', class_='sr-only').get_text().strip()
            text_price1 = old_price.replace("$", "")
            text_price2 = text_price1.replace("Your price for this item is ", "")
            nocommas = text_price2.replace(",", "")
            price = float(nocommas)
            
            partial_link = mother.find('div', class_='sku-title').find('a')['href'].strip()
            base_url = "https://www.bestbuy.com/"
            link = base_url + partial_link
            model = mother.find('span', class_='sku-value').get_text().strip()
            sku = soup.find('div', class_='sku-model').findAll('div', class_='sku-attribute-title')[1].find('span', class_='sku-value').get_text().strip()
            title_nospace = title.replace(" ", "")
            title_to_list = title_nospace.split("-")

        dic = {
            'title': title,
            'price': price,
            'link': link,
            'upc': sku,
            'model': model,
            'brand' : title_to_list[0]
        }
        bestbuy_laptops.append(dic)

    except AttributeError:
        print("error found, skipping...")
        print(page_no)
        print(title)
        
    
    page_no += 1
    next_page = f'https://www.bestbuy.com/site/windows/windows-laptops/pcmcat748300547900.c?cp={page_no}&id=pcmcat748300547900'
    browser.visit(next_page)
    time.sleep(1.5)

    htmltwo = browser.html
    soup = bs(htmltwo, 'html.parser') 
    
    next_page_avail = soup.find_all('h1')
    for h in next_page_avail:
        message = h.text
        if message == "We’re sorry, something went wrong.":
            page_remaining = False





error found, skipping...
2
Microsoft - Surface Pro 6 - 12.3" Touch-Screen - Intel Core i7 - 16GB Memory - 512GB Solid State Drive (Latest Model) - Black
error found, skipping...
3
Microsoft - Surface Pro 6 - 12.3" Touch-Screen - Intel Core i5 - 8GB Memory - 256GB Solid State Drive (Latest Model) - Black
error found, skipping...
4
Microsoft - Surface Pro 6 - 12.3" Touch-Screen - Intel Core i7 - 8GB Memory - 256GB Solid State Drive (Latest Model) - Black


In [ ]:
###############################################################
# LOAD: 
###############################################################
# create a dataframe from the list of dictionaries:
frys_df = pd.DataFrame(frys_laptops)
frys_df.head()

bestbuy_df = pd.DataFrame(bestbuy_laptops)
bestbuy_df.head()

# change to your username:password@localhost:5432/laptop_db:
connection = "postgres:helloworld@localhost:5432/laptop_db"
engine = create_engine(f'postgresql://{connection}')

# add the FRYS dataframe to the table:
frys_df.to_sql(name="frys_laptops", con=engine, if_exists='replace', index=False)



# add the BEST BUY dataframe to the table:
bestbuy_df.to_sql(name="bestbuy_laptops", con=engine, if_exists='replace', index=False)


# VALIDATE TABLE BUILD:
engine.table_names()